In [48]:
from time import gmtime, strftime
from tqdm import tqdm
import importlib

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output
import IPython
# %matplotlib qt
import seaborn as sns

# # import stream_generator
# # import learning_models
# # from datasets.data_loader import load_dataset
# import dth
# from baselines_v2 import kswin_reg
# from baselines_v2 import naive_reg
# from baselines_v2 import adwin_reg
# import msmsa_v3 as msmsa


# from sklearn.ensemble import RandomForestRegressor
# from sklearn.tree import DecisionTreeRegressor

# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler


# import neural_net_base_learner
# # import wandb
# import os
# from utility.utilities import Logger, Plotter
# from utility.sample import Sample





def generate_2d_gaussian_random_field(num_points, scale):
    # Sample x and y randomly
    x = np.random.uniform(0, 1, num_points)
    t = np.random.uniform(0, 1, num_points)

    # append the corners
    x = np.append(x, [0, 0, 1, 1])
    t = np.append(t, [0, 1, 0, 1])

    # adjust the number of points
    num_points = len(x)


    
    # Compute the distance matrix
    d = np.sqrt((x[:, None] - x[None, :])**2 + (t[:, None] - t[None, :])**2)
    
    # Define the covariance function
    covariance_func = np.exp(-d**2 / (2 * scale**2))
    
    # Generate the random field
    mean = np.zeros(num_points)
    y = np.random.multivariate_normal(mean, covariance_func)
    y = y + np.random.normal(0, 0, num_points)
    return x, t, y

def generate_2_class_field(num_points):
    # Sample x and y randomly
    x = np.random.uniform(0, 1, num_points)
    t = np.random.uniform(0, 1, num_points)
    
    # Generate the random field
    y = np.ones(num_points)
    for i in range(num_points):
        if 2*x[i] - int(2*x[i]) > 0.5 and  0.2 < t[i] < 0.4:
        # if x[i]+t[i] > 1:
            y[i] = -1

    y = y + np.random.normal(0, .0, num_points)
    return x, t, y

def generate_2d_abrupt_drift_field(num_points):
    # Sample x and y randomly
    x = np.random.uniform(1, 2, num_points)
    t = np.random.uniform(0, 1, num_points)
    
    # Generate the random field
    y = np.ones(num_points)
    for i in range(num_points):
        if t[i] > 0.5 and x[i] < 1:
        # if t[i] > 0.5:
        # if x[i]+t[i] > 1:
            y[i] = -1
            # print(y[i],i)
    y = y + np.random.normal(0, 0.3, num_points)
    return x, t, y


# Parameters
num_points = 1000 # Number of points to sample
scale = 0.2  # Scale of the correlation

# Generate field
# x1, t1, y1 = generate_2d_gaussian_random_field(num_points, scale)
# x2, t2, y2 = generate_2_class_field(int(num_points/0.5))
x2, t2, y2 = generate_2d_gaussian_random_field(num_points, scale)


# # combine the data into one dataset
# x = np.concatenate((x1, x2+1))
# t = np.concatenate((t1, t2))
# y = np.concatenate((y1, y2))


x, t, y = generate_2d_abrupt_drift_field(num_points)



# x = x1
# t = t1
# y = y1

x = x2
t = t2
y = y2

num_samples = len(x)
# Make a stream of samples
stream = []
for i in range(num_samples):
    stream.append((x[i], y[i], t[i], None))
# sort samples in the stream by time
stream.sort(key=lambda sample: sample[2])




In [54]:

%matplotlib qt
# Create a contour plot
plt.figure(figsize=(5, 4))
# plt.tricontourf(x, t, y, levels=20, cmap='viridis')
# plt.colorbar(label='Target attribute y')
vmin = min(y)
vmax = max(y)
# scatter plot using sns

# change the font to sans-serif

# close the previous plot
plt.close('all')

# make a colorbar

plt.rcParams["font.family"] = "Microsoft Sans Serif"
# use searborn to plot kde contourf plot on the background
# plt.tricontourf(x, t, y, levels=100, cmap='viridis', vmin=vmin, vmax=vmax, alpha=0.4, antialiased=True)

# randomly sample 500 points to plot
idx = np.random.choice(num_samples, 500, replace=False)
x_ = x[idx]
t_ = t[idx]
y_ = y[idx]

# add some noise
x_ = x_ + np.random.normal(0, 0.1, len(x_))

plt.scatter(x_, t_, c=y_, s=20, cmap='jet',vmin=vmin, vmax=vmax, edgecolor='g', linewidth=0.5)
plt.colorbar(label='Target attribute y')





plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('Gaussian Random Field Stream')
plt.xlabel('x')
plt.ylabel('Time')
plt.show()
